## TODO: Build our own MCP, only useful if we want to share it with somebody else, exploration purpose

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio

from IPython.display import display, Markdown


load_dotenv()

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=7960.218580000001, strategy='', holdings={'AMZN': 9}, transactions=[3 shares of AMZN at 226.64238 each., 3 shares of AMZN at 226.64238 each., 3 shares of AMZN at 226.64238 each.], portfolio_value_time_series=[('2025-12-13 17:09:28', 9998.64286), ('2025-12-13 17:09:51', 9998.64286), ('2025-12-13 17:46:29', 9997.28572), ('2025-12-13 17:46:30', 9997.28572), ('2025-12-13 17:58:59', 9995.92858), ('2025-12-13 17:58:59', 9995.92858)])

In [4]:
account.buy_shares("AMZN", 3, "because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 7960.218580000001, "strategy": "", "holdings": {"AMZN": 9}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 226.64238, "timestamp": "2025-12-13 17:09:28", "rationale": "because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 226.64238, "timestamp": "2025-12-13 17:46:29", "rationale": "because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 226.64238, "timestamp": "2025-12-13 17:58:58", "rationale": "because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-13 17:09:28", 9998.64286], ["2025-12-13 17:09:51", 9998.64286], ["2025-12-13 17:46:29", 9997.28572], ["2025-12-13 17:46:30", 9997.28572], ["2025-12-13 17:58:59", 9995.92858]], "total_portfolio_value": 9995.92858, "total_profit_loss": -4.071420000001126}'

In [5]:
account.report()

'{"name": "ed", "balance": 7960.218580000001, "strategy": "", "holdings": {"AMZN": 9}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 226.64238, "timestamp": "2025-12-13 17:09:28", "rationale": "because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 226.64238, "timestamp": "2025-12-13 17:46:29", "rationale": "because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 226.64238, "timestamp": "2025-12-13 17:58:58", "rationale": "because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-13 17:09:28", 9998.64286], ["2025-12-13 17:09:51", 9998.64286], ["2025-12-13 17:46:29", 9997.28572], ["2025-12-13 17:46:30", 9997.28572], ["2025-12-13 17:58:59", 9995.92858], ["2025-12-13 17:58:59", 9995.92858]], "total_portfolio_value": 9995.92858, "total_profit_loss": -4.071420000001126}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 226.64238,
  'timestamp': '2025-12-13 17:09:28',
  'rationale': 'because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 226.64238,
  'timestamp': '2025-12-13 17:46:29',
  'rationale': 'because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 226.64238,
  'timestamp': '2025-12-13 17:58:58',
  'rationale': 'because this bookstore website looks promising'}]

# Write MCP server and use it directly

In [5]:
import subprocess, time, select

p = subprocess.Popen(
    ["uv", "run", "accounts_server.py"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1,
)

time.sleep(0.5)

def read_available(pipe, limit=500):
    if pipe is None:
        return ""
    ready, _, _ = select.select([pipe], [], [], 0)
    if not ready:
        return ""
    return pipe.read(limit)

out = read_available(p.stdout, 500)
err = read_available(p.stderr, 2000)

print("alive:", p.poll() is None)
print("stdout head:", repr(out))
print("stderr head:", repr(err))

alive: True
stdout head: ''
stderr head: ''


In [4]:
params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

Error initializing MCP server: Connection closed


McpError: Connection closed

In [7]:
import os, sys
from pathlib import Path
from agents.mcp import MCPServerStdio

params = {
    "command": sys.executable,  # or "uv"
    "args": ["accounts_server.py"],  # or ["run", "accounts_server.py"] if you prefer uv
    "cwd": str(Path(".").resolve()),
    "env": dict(os.environ),  # keep VIRTUAL_ENV, PYTHONPATH, etc.
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given accont name\n        args: \n            name: name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None),
 Tool(name='get_holdings', title=None, description='Get holdings of the given account name\n    \n    Args: \n        name: the name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution